### Code used for converting the array matrix results into GeoTIFs for Remote Sensing visualizations

#### After the analysis using the matrices in Python, the results might be explored in a GIS format, therefore this code provides the further conversion of the main results from the ARRAY format into GeoTif


Developed by: Thiago Victor Medeiros do Nascimento

In [1]:
from pcraster import *
import numpy as np
from osgeo import gdal, gdalconst
from osgeo import gdal_array
from osgeo import osr
import matplotlib.pylab as plt
import subprocess
import glob,os
import pandas as pd

In [2]:
# This is a clone map used just to import the geotrasnformation information for the rasters creation.
pathclone =r'D:\pythonDATA\AquadaptTemez\rastermapaccum\T1accum.map'
RasterClone = gdal.Open(pathclone)

geotransform = RasterClone.GetGeoTransform()

ncols = RasterClone.RasterXSize
nrows = RasterClone.RasterYSize
numtotal = ncols*nrows
xmin = RasterClone.GetGeoTransform()[0]
ymax = RasterClone.GetGeoTransform()[3]
xres = RasterClone.GetGeoTransform()[1]
yres = RasterClone.GetGeoTransform()[5]
xmax = xmin + xres*ncols
ymin = ymax - xres*nrows

print("These are the main information for our rasters:", geotransform)
print("The number of rows is:", nrows, "and the number of columns is:", ncols)

These are the main information for our rasters: (-118821.6376, 100.0, 0.0, 418121.532, 0.0, -100.0)
The number of rows is: 7140 and the number of columns is: 6820


In [21]:
# Importation of the arrays used for the calculations::

path =r'C:\Users\User\OneDrive\IST\RESEARCH\1_flowindicators\gis\csvs\nummonzerrunperc25.csv'
resmatrix=pd.read_table(path, header = None)   
    
resarray = np.array(resmatrix)
resarray = resarray.T

resarray

array([[2.5, 2.5, 2.5, ..., 4. , 4. , 4. ]])

In [22]:
resmatrix

,0
0,2.5
1,2.5
2,2.5
3,2.5
4,2.5
...,...
578970,4.0
578971,4.0
578972,4.0
578973,4.0


In [23]:
np.nanmax(resmatrix)

11.0

In [24]:
# At this point we have to recriate the complete array, since it was filtered for the computation of the indicators, then:

# First we read our previous used filter map:
#pathfilter =r'C:\Users\User\OneDrive\IST\RESEARCH\python\flowindicatorsmap\rivernetworkabove50000mmclipped2.map'
pathfilter =r'C:\Users\User\OneDrive\IST\RESEARCH\python\flowindicatorsmap\rivernetworkabove100000mmclipped.map'

# MK, 2,4 and 5: 50,000
# 1 and 3: 100,000

mapfilter = readmap(pathfilter)
Rastermapfilter = gdal.Open(pathfilter)
mapfilterarray = pcr_as_numpy(mapfilter)
mapfilterarray
mapfilterarray[mapfilterarray < 1 ] = np.nan

# We need additionally to reshape our filter:
mapfilterarrayres = np.reshape(mapfilterarray, (1, numtotal))

In [25]:
# We first create a matrix with the total dimensions of the original raster:
resultarraytotal = np.zeros((1,numtotal),dtype=np.float32)
resultarraytotal[:,:] = np.nan
#resultarraytotal[:,:] = -9999 # We can use -9999 as NaN values

In [26]:
# Now we reasign our data for the non NaNs of this matrix, an reshape it for a matrix:
resultarraytotal[~np.isnan(mapfilterarrayres)] = resarray[0,:]

resultmatrixtotal = np.reshape(resultarraytotal, (nrows, ncols))

In [27]:
# Just to check it out:
resultmatrixtotal.shape

(7140, 6820)

In [28]:
# We create a function to write down the MATRIX into a Raster:

def WriteFromArrayToGTIFF(resultmatrixtotal, geotransform, pathoutput):

    output_raster = gdal.GetDriverByName('GTiff').Create(pathoutput,ncols, nrows, 1 ,gdal.GDT_Float32)  # Open the file
    output_raster.SetGeoTransform(geotransform)  # Specify its coordinates
    srs = osr.SpatialReference()                 # Establish its coordinate encoding
    srs.ImportFromEPSG(3763)                                                                                                               
    output_raster.SetProjection( srs.ExportToWkt() )   # Exports the coordinate system 
                                                   # to the file
    output_raster.GetRasterBand(1).WriteArray(resultmatrixtotal)   # Writes my array to the raster

    output_raster.FlushCache()

In [29]:
# Finally we write down the results as a Raster file (Tif)
pathoutput = r'C:\Users\User\OneDrive\IST\RESEARCH\1_flowindicators\gis\rasters\nummonzerrunperc25.tif'
WriteFromArrayToGTIFF(resultmatrixtotal, geotransform, pathoutput)